In [29]:
import polars as pl

#we need to tell polars to treat 'NA' as 'Null' so it doesn't get confused between text and numbers
df = pl.read_csv('../Data/train.csv',
                 null_values='NA',
                infer_schema_length=1000 
)

#check the size of our data
print(f'The dataset contains {df.height} rows and {df.width} columns.')

#identify the columns with missing values
#create a list of columns and their missing value counts
null_summary = df.null_count().melt().filter(pl.col('value') > 0).sort('value', descending=True)

#view the list without truncation
with pl.Config(tbl_rows=100):
    print(null_summary)

#create a list of the 'Ghost' columns we want to drop, then drop them.
ghost_columns = ['PoolQC', 'MiscFeature', 'Alley', 'Fence']

df_reduced = df.drop(ghost_columns)
print(f'New column count: {df_reduced.width}')

#Columns where Null means Feature not present
structural_cols = [
    'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'
]

#Fill Nulls with None
df_structural = df_reduced.with_columns([
    pl.col(col).fill_null('None') for col in structural_cols
])

#Verify no Nulls remain in these columns
print(f'Nulls in FireplaceQu after fix: {df_structural["FireplaceQu"].null_count()}')

# Calculate the median of the street frontage
median_value = df_structural["LotFrontage"].median()

# Fill the holes with that median
df_final_audit = df_structural.with_columns(
    pl.col("LotFrontage").fill_null(median_value)
)

print(f"LotFrontage median used: {median_value}")
print(f"Total remaining nulls in dataset: {df_final_audit.null_count().sum().sum()}")


#we now have a fairly solid dataset to work with.
#which columns dictate the price?
numeric_df = df_final_audit.select(pl.col(pl.Int64, pl.Float64)) #select numeric columns for correlation analysis

corr_matrix = numeric_df.corr()

price_correlations = (
    corr_matrix.with_columns(
        pl.Series("Feature", numeric_df.columns)
    )
    .select([
        pl.col("Feature"),
        pl.col("SalePrice")
    ])
    .sort("SalePrice", descending=True)
)

with pl.Config(tbl_rows=150):
    print(price_correlations)

#checking for independance
garage_overlap = numeric_df.select(pl.corr("GarageArea", "GarageCars"))
size_overlap = numeric_df.select(pl.corr('GrLivArea', 'TotalBsmtSF'))

print(f'Garage overlap: {garage_overlap[0,0]:.4f}')
print(f'Size overlap: {size_overlap[0,0]:.4f}')

#drop GarageArea since it overlaps heavily with GarageCars.
df_final_audit = df_final_audit.drop('GarageArea')

#Search for outliers.
# Filter for houses that are huge (GrLivArea > 4000)
outliers = df_final_audit.filter(pl.col("GrLivArea") > 4000)

# Let's see their Price vs their Size
print("Potential Outliers (Huge Houses):")
print(outliers.select(["Id", "GrLivArea", "SalePrice", "OverallQual"]))

# Remove the outliers by keeping only houses under 4000 sq ft
df_final_audit = df_final_audit.filter(pl.col("GrLivArea") < 4000)

print(f"Outliers removed. Final row count: {df_final_audit.height}")

# Save to the data folder so we can find it easily tomorrow
df_final_audit.write_csv("../Data/train_cleaned.csv")

print("File saved successfully as 'train_cleaned.csv' in your data folder!")


The dataset contains 1460 rows and 81 columns.
shape: (19, 2)
┌──────────────┬───────┐
│ variable     ┆ value │
│ ---          ┆ ---   │
│ str          ┆ u32   │
╞══════════════╪═══════╡
│ PoolQC       ┆ 1453  │
│ MiscFeature  ┆ 1406  │
│ Alley        ┆ 1369  │
│ Fence        ┆ 1179  │
│ FireplaceQu  ┆ 690   │
│ LotFrontage  ┆ 259   │
│ GarageType   ┆ 81    │
│ GarageYrBlt  ┆ 81    │
│ GarageFinish ┆ 81    │
│ GarageQual   ┆ 81    │
│ GarageCond   ┆ 81    │
│ BsmtExposure ┆ 38    │
│ BsmtFinType2 ┆ 38    │
│ BsmtQual     ┆ 37    │
│ BsmtCond     ┆ 37    │
│ BsmtFinType1 ┆ 37    │
│ MasVnrType   ┆ 8     │
│ MasVnrArea   ┆ 8     │
│ Electrical   ┆ 1     │
└──────────────┴───────┘
New column count: 77
Nulls in FireplaceQu after fix: 0
LotFrontage median used: 69.0
Total remaining nulls in dataset: shape: (1, 77)
┌─────┬────────────┬──────────┬─────────────┬───┬────────┬──────────┬───────────────┬───────────┐
│ Id  ┆ MSSubClass ┆ MSZoning ┆ LotFrontage ┆ … ┆ YrSold ┆ SaleType ┆ SaleConditi

C:\Users\alist\AppData\Local\Temp\ipykernel_27548\3845587670.py:14: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  null_summary = df.null_count().melt().filter(pl.col('value') > 0).sort('value', descending=True)
